## 1. Описание проекта

Задачей проекта является построение скоринг-модели для вторичных клиентов банка, которая бы предсказывала вероятность дефолта клиента.  
Для этого нужно определить значимые параметры заемщика.  
Результаты оцениваются по Mean F1-Score

In [ ]:
# импортируем необходимые библиотеки

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import f_classif, mutual_info_classif
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold, StratifiedShuffleSplit
from sklearn.model_selection import cross_val_score, cross_validate, GridSearchCV
from sklearn.metrics import mean_squared_error, f1_score, accuracy_score
from sklearn.metrics import balanced_accuracy_score, precision_recall_curve
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, roc_auc_score
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler, LabelEncoder, OneHotEncoder
from itertools import combinations, combinations_with_replacement

%matplotlib inline
import warnings
warnings.simplefilter('ignore')

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
!pip install -U scikit-learn

## 2. Сбор и знакомство с данными

In [ ]:
# загружаем данные
DATA_DIR = '/kaggle/input/sf-scoring/'
df_train = pd.read_csv(DATA_DIR +'/train.csv')
df_test = pd.read_csv(DATA_DIR +'/test.csv')
sample_submission = pd.read_csv(DATA_DIR+'/sample_submission.csv')

In [ ]:
print(f'shape of df_train: {df_train.shape}')
print(f'shape of df_test: {df_test.shape}')
print(f'shape of sample_submission: {sample_submission.shape}')

In [ ]:
df_train.tail()

In [ ]:
df_test.tail()

In [ ]:
sample_submission.tail()

In [ ]:
# проверим идентичность client_id в тестовой и submission выборках
(df_test.client_id == sample_submission.client_id).value_counts()

In [ ]:
# ВАЖНО! дря корректной обработки признаков объединяем трейн и тест в один датасет
df_train['sample'] = 1 # помечаем где у нас трейн
df_test['sample'] = 0  # помечаем где у нас тест
df_test['default'] = -1 # в тесте у нас нет значения Rating, мы его должны предсказать, по этому пока просто заполняем невозможным значением -1

data = df_test.append(df_train, sort=False).reset_index(drop=True) # объединяем
data.shape

In [ ]:
# Приглядимся, а что собственно за данные у нас есть
data.info()

In [ ]:
data.nunique(dropna=False)

### Описание полей

- client_id - идентификатор клиента  
- app_date - дата подачи заявки
- education - уровень образования
- sex - пол
- age - возраст
- car - наличие автомобиля
- car_type - наличие определенного автомобиля, например стоимостью более 1 млн.руб.
- decline_app_cnt - количество отклоненных заявок
- good_work - наличия хорошей работы
- score_bki - скоринговый балл по данным из БКИ
- bki_request_cnt - количество запросов в БКИ
- region_rating - рейтинг региона
- home_address - категория домашнего адреса, например определенный район или удаленность от центра
- work_address - категория рабочего адреса, аналогично
- income - доход
- sna - связь заемщика с клиентами банка (что это значит, не представляю...)
- first_time - давность наличия информации о заемщике
- foreign_passport - наличие загранпаспорта
- default - флаг дефолта по кредиту (0 - "хороший клиент", 1 - "плохой клиент")

In [ ]:
# посмотрим распределение целевой переменной в обучающей выборке
percentage = '{:.1f}%'.format(100*df_train['default'].sum()/len(df_train['default']))
print(f"Доля неплатежеспособных заемщиков составляет {percentage} от общего количества наблюдений в обучающей выборке.\n")
sns.countplot(x='default', data=df_train)
plt.show()

Специалисты данной области рекомендуют для построения скоринговых моделей использовать выборку с долей неплатежеспособных заемщиков не менее 5%.  
В нашем случае эта доля составляет 12,7%, это хорошо. Но в любом случае нужно учесть несбалансированность классов.

## 3. Очистка и предобработка данных

In [ ]:
# посмотрим на пропуски
data.isna().sum()

Решим проблему с пропусками, когда доберемся до этого признака.

### 3.1 Client id

In [ ]:
# проверим, есть ли дублирование данных по client id
print(f"Все клиенты уникальны: {data['client_id'].nunique() == data.shape[0]}")

In [ ]:
# проверим, является ли client_id ординальным признаком. Если да, то должен быть высоким коэф.корреляции с датой подачи заявок
# сначала обработаем признак app_date
data.app_date = pd.to_datetime(data.app_date)

In [ ]:
# проверим app_date на адекватность
data.app_date.hist(bins=data.app_date.nunique(), xrot=45, grid=False)
data.app_date.value_counts()

Выборосов или неадекватных временных меток не обнаружено. Все данные более менее равномерно распределены в промежутке с 1 января 2014 по 30 апреля 2014.  
Есть предположение, что в выходные дни количество заявок резко проседает.

In [ ]:
data['ts'] = data['app_date'].astype('int64') // 10**9
data[['client_id','ts']].corr()

Гипотеза об ординальности признака client_id и соответственно о тесной связи с app_date подтвердилась.  
Логической нагрузки признак client_id не несет, поэтому удалим его.

In [ ]:
data.drop('client_id', axis=1, inplace=True)

### 3.2 App_date

In [ ]:
# интересно посмотреть, есть ли тенденция в количестве дефолтов в зависимости от времени
sns.displot({"default 0": data[data['default'] == 0].ts,
             "default 1": data[data['default'] == 1].ts,
             },
            kde=True,
            common_norm=True # независимая нормализация каждого подмножества
            )
plt.title('App date', fontsize=20)
plt.xlabel('App date', fontsize=14)
plt.ylabel('Dentsity', fontsize=14)
#plt.grid()

plt.xticks()
plt.yticks(fontsize=14);

Монотонного роста или падения количества дефолтов  зависимости от времени не наблюдается.  
Из признака app_date я выделю новые признаки: месяц, день недели и признак выходного дня.

In [ ]:
data['app_month']=data['app_date'].dt.month
data['app_day_of_week']=data['app_date'].dt.day_of_week
data['app_is_day_off']=data['app_day_of_week'].apply(lambda x: 1 if x>4 else 0)

In [ ]:
# признаки app_date и ts (timestamp) удалим
data.drop(['app_date','ts'], axis=1, inplace=True)

### 3.3 Education

In [ ]:
data.education.unique()

SCH - school, школа, 8 классов

GRD - graduate, колледж, 10-12 классов

UGR - undergraduate, ВУЗ

PGR - postgraduate, аспирант

ACD - academy, профессор

In [ ]:
data.education.value_counts(dropna=False)

In [ ]:
# пропущенных значений относительно общего объема выборки не велик. Но с другой стороны, профессоров еще меньше.
# чтобы не потерять информацию о возможном осознанном утаивании оставим пропуски как еще одно значение данного признака
data.education.fillna("unknown", inplace=True)

### 3.4 Sex, car, car_type, foreign_passport

In [ ]:
# преобразуем эти признаки в числовые для более удобной дальнейшей обработки
bin_cols = ['sex', 'car', 'car_type', 'foreign_passport']
le = LabelEncoder()
for col in bin_cols:
    data[col] = le.fit_transform(data[col])

In [ ]:
data.info()

### 3.5 Age

In [ ]:
# построим графики распределения переменной возраст, общий и в разрезе целевой переменной
sns.distplot(data[data['sample'] == 1].age, kde=False)
sns.distplot(data[data.default == 0].age, kde=False, label='default 0')
sns.distplot(data[data.default == 1].age, kde=False, label='default 1')
plt.legend()
plt.show()

Видим логнормальное распределение. Выбросов нет.

## 4. Монотонный WOE binning переменных

In [ ]:
# функция для биннинга категориальных и непрерывных переменных и вычисления WOE/IV
# честно заимствована отсюда https://www.listendata.com/2015/03/weight-of-evidence-woe-and-information.html
def iv_woe(data, target, bins=10, show_woe=False):
    '''Function calculates IV (Information value) and WoE (weight of evidence)
    for categorial and numeric features.'''    
        
    #Empty Dataframe
    newDF,woeDF = pd.DataFrame(), pd.DataFrame()
    
    #Extract Column Names
    cols = data.columns
    
    #Run WOE and IV on all the independent variables
    for ivars in cols[~cols.isin([target])]:
        if (data[ivars].dtype.kind in 'bifc') and (len(np.unique(data[ivars]))>bins):
            binned_x = pd.qcut(data[ivars], bins,  duplicates='drop')
            d0 = pd.DataFrame({'x': binned_x, 'y': data[target]})
        else:
            d0 = pd.DataFrame({'x': data[ivars], 'y': data[target]})
        d = d0.groupby("x", as_index=False).agg({"y": ["count", "sum"]})
        d.columns = ['Cutoff', 'N', 'Events']
        d['% of Events'] = np.maximum(d['Events'], 0.5) / d['Events'].sum()
        d['Non-Events'] = d['N'] - d['Events']
        d['% of Non-Events'] = np.maximum(d['Non-Events'], 0.5) / d['Non-Events'].sum()
        d['WoE'] = np.log(d['% of Events']/d['% of Non-Events'])
        d['IV'] = d['WoE'] * (d['% of Events'] - d['% of Non-Events'])
        d.insert(loc=0, column='Variable', value=ivars)
        #print("Information value of " + ivars + " is " + str(round(d['IV'].sum(),6)))
        temp =pd.DataFrame({"Variable" : [ivars], "IV" : [d['IV'].sum()]}, columns = ["Variable", "IV"])
        newDF=pd.concat([newDF,temp], axis=0)
        woeDF=pd.concat([woeDF,d], axis=0)

        #Show WOE Table
        if show_woe == True:
            print(d)
    newDF = newDF.sort_values(by='IV', ascending=False).reset_index(drop=True)
    woeDF.reset_index(drop=True, inplace=True)
    return newDF, woeDF

In [ ]:
# функция для построения графиков категоризованных признаков и WOE
def plot_bin(woeDF, variable):
    tmp = woeDF[woeDF.Variable==variable]
    fig,ax = plt.subplots(figsize=(16,8), facecolor='w')
    ax.bar(tmp.Cutoff.astype(str), tmp.WoE)
    ax.plot(tmp.Cutoff.astype(str), tmp.WoE, color='r', label=variable)
    ax.yaxis.grid()
    plt.tight_layout()
    plt.legend()
    plt.show()

In [ ]:
newDF, woeDF = iv_woe(data[data['sample']==1], 'default', bins=10, show_woe=False)

In [ ]:
newDF

In [ ]:
woeDF

### 5. Отсев переменных по information value, ручной binning

Первое, что нужно сделать - отсеять признаки с IV < 0.02, при этом по возможности сохранить исходные данные.

In [ ]:
# Посмотрим на матрицу корреляций
plt.figure(figsize=(16, 10))
sns.heatmap(data.corr().abs(), annot=True, cmap='coolwarm', fmt='.3f', annot_kws={'size':10})
plt.show()

In [ ]:
woeDF[woeDF.Variable.isin(['app_is_day_off','app_day_of_week'])]

In [ ]:
# ввиду низкой предиктивной силы, высокой корреляции и исскуственной синтезации признаков 'app_is_day_off','app_day_of_week', удалим их
data.drop(['app_is_day_off','app_day_of_week', 'app_month'], axis=1, inplace=True)

In [ ]:
plot_bin(woeDF, 'age')

In [ ]:
woeDF[woeDF.Variable=='age']

In [ ]:
data['binned_age'] = pd.cut(data.age, [20,26,31,37,51,72])

In [ ]:
newDF, woeDF = iv_woe(data[data['sample']==1], 'default', bins=10, show_woe=False)
newDF

In [ ]:
plot_bin(woeDF, 'binned_age')

In [ ]:
woeDF[woeDF.Variable=='binned_age']

Есть линейная связь между binned_age и целевой переменной, но предсказательная сила очень мала

In [ ]:
data.drop('age', axis=1, inplace=True)

Попробуем объединить признаки car и car_type в один

In [ ]:
data.groupby('car').car_type.value_counts()

In [ ]:
# просто сложим два признака.
data['mapped_car'] = data['car'] + data['car_type']

In [ ]:
newDF, woeDF = iv_woe(data[data['sample']==1], 'default', bins=10, show_woe=False)
newDF

In [ ]:
# отлично, признаки по отдельности удалим
data.drop(['car','car_type'], axis=1, inplace=True)

Проделаем ту же хитрость с адресами

In [ ]:
data.groupby(['home_address'])['work_address'].value_counts().rename('count').reset_index()

In [ ]:
plot_bin(woeDF, 'work_address')

In [ ]:
data['home_work_address'] = (data.home_address.astype(str)+data.work_address.astype(str)).astype(int)

In [ ]:
data['home_work_address'].value_counts()

In [ ]:
newDF, woeDF = iv_woe(data[data['sample']==1], 'default', bins=10, show_woe=False)
newDF

In [ ]:
woeDF[woeDF.Variable=='home_work_address']

In [ ]:
plot_bin(woeDF.sort_values('WoE'), 'home_work_address')

In [ ]:
data.drop(['home_address','work_address'], axis=1, inplace=True)

In [ ]:
data['good_work_sex'] = (data.good_work.astype(str)+data.sex.astype(str)).astype(int)
newDF, woeDF = iv_woe(data[data['sample']==1], 'default', bins=10, show_woe=False)
newDF

In [ ]:
data.drop(['good_work','sex'], axis=1, inplace=True)

In [ ]:
plot_bin(woeDF, 'income')

In [ ]:
data['binned_income'] = pd.cut(data.income, [999,22000,35000,40000,70000,1000000])

In [ ]:
newDF, woeDF = iv_woe(data[data['sample']==1], 'default', bins=10, show_woe=False)
newDF

In [ ]:
plot_bin(woeDF, 'binned_income')

In [ ]:
data.drop('income', axis=1, inplace=True)

In [ ]:
plt.figure(figsize=(16, 8))
sns.heatmap(data.corr().abs(), annot=True, cmap='coolwarm', fmt='.3f', annot_kws={'size':10})

In [ ]:
data.groupby(['first_time'])['sna'].value_counts().rename('percentage').reset_index()

In [ ]:
plot_bin(woeDF, 'first_time')

In [ ]:
plot_bin(woeDF, 'sna')

In [ ]:
data['first_time_sna'] = data.first_time/data.sna

In [ ]:
newDF, woeDF = iv_woe(data[data['sample']==1], 'default', bins=10, show_woe=False)
newDF

In [ ]:
plot_bin(woeDF, 'first_time_sna')

In [ ]:
data.drop(['first_time','sna'], axis=1, inplace=True)

In [ ]:
data.nunique()

In [ ]:
plot_bin(woeDF, 'bki_request_cnt')

In [ ]:
data['binned_bki_request_cnt'] = pd.qcut(data['bki_request_cnt'], 10,  duplicates='drop')
data.drop('bki_request_cnt', axis=1, inplace=True)

In [ ]:
data['binned_bki_request_cnt'].value_counts()

In [ ]:
plot_bin(woeDF, 'score_bki')

In [ ]:
data['binned_score_bki'] = pd.qcut(data['score_bki'], 10,  duplicates='drop')
data.drop('score_bki', axis=1, inplace=True)

In [ ]:
plot_bin(woeDF, 'decline_app_cnt')

In [ ]:
woeDF[woeDF.Variable=='decline_app_cnt']

In [ ]:
data.decline_app_cnt.value_counts()

In [ ]:
data['binned_decline_app_cnt'] = pd.qcut(data['decline_app_cnt'], 10,  duplicates='drop')
data.drop('decline_app_cnt', axis=1, inplace=True)

In [ ]:
plot_bin(woeDF, 'region_rating')

In [ ]:
newDF, woeDF = iv_woe(data[data['sample']==1], 'default', bins=10, show_woe=False)
newDF

In [ ]:
data.nunique()

In [ ]:
data = pd.get_dummies(data)

In [ ]:
data.head()

In [ ]:
data.info()

In [ ]:
# Помимо отбора по IV добавим рекурсивный поиск оптимального количества переменных методом RFE из sklearn.
from sklearn.feature_selection import RFECV

def RFE_feature_selection(clf_lr, X, y):
    rfecv = RFECV(estimator=clf_lr, step=1, cv=StratifiedKFold(5), verbose=0, scoring='roc_auc')
    rfecv.fit(X, y)

    print("Optimal number of features : %d" % rfecv.n_features_)

    # Plot number of features VS. cross-validation scores
    f, ax = plt.subplots(figsize=(14, 9))
    plt.xlabel("Number of features selected")
    plt.ylabel("Cross validation score (nb of correct classifications)")
    plt.plot(range(1, len(rfecv.grid_scores_) + 1), rfecv.grid_scores_)
    plt.show()
    mask = rfecv.get_support()
    #X = X.idx[:, mask]
    return mask

In [ ]:
clf_lr = LogisticRegression(solver='liblinear')
X = data[data['sample']==1].drop(['sample', 'default'], axis=1)
y = data[data['sample']==1]['default']
mask = RFE_feature_selection(clf_lr, X, y)
mask

In [ ]:
X.columns[~mask]

In [ ]:
# Теперь выделим тестовую часть
train_data = data.query('sample == 1').drop(['sample'], axis=1)
test_data = data.query('sample == 0').drop(['sample', 'default'], axis=1)

y = train_data['default'].values  # наш таргет
X = train_data.drop(['default'], axis=1)

In [ ]:
# Воспользуемся специальной функцие train_test_split для разбивки тестовых данных
from sklearn.model_selection import train_test_split

# выделим 20% данных на валидацию (параметр test_size)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# проверяем
X_train.shape, X_test.shape, y_train.shape, y_test.shape

In [ ]:
# Импортируем необходимые библиотеки:
from sklearn.linear_model import LogisticRegression # инструмент для создания и обучения модели
from sklearn import metrics # инструменты для оценки точности модели

In [ ]:
logreg = LogisticRegression(solver='liblinear', max_iter=1000)
logreg.fit(X_train, y_train)
y_pred = logreg.predict(X_test)

In [ ]:
from sklearn.metrics import classification_report
classification_report = classification_report(y_test, y_pred)
print(classification_report)

In [ ]:
# если качество нас устраивает, обучаем финальную модель на всех обучающи данных
logreg_final = LogisticRegression(solver='liblinear', max_iter=1000)
logreg_final.fit(X, y)

In [ ]:
predict_submission = logreg_final.predict(test_data)

In [ ]:
sample_submission['default'] = predict_submission
sample_submission.to_csv('submission.csv', index=False)
sample_submission.head(10)

In [ ]:
sample_submission.describe()

In [ ]:
#!kaggle competitions submit -c sf-scoring -f ssubmission.csv -m "Message"
# !kaggle competitions submit your-competition-name -f submission.csv -m 'My submission message'